**Import file training dari Kaggle input**

In [ ]:
import os
from sklearn.metrics import classification_report, confusion_matrix
try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


**Import Kebutuhan Plugin**

In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,classification_report,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
import itertools
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

**Deklarasi PATH gambar**

In [ ]:
X = []
Z = []
IMG_SIZE = 75
pisang_matang = '../input/datapisang/pisang/pisang matang'
pisang_mentah = '../input/datapisang/pisang/pisang mentah'

**Membuat variable**

In [ ]:
def assign_label(img,maks):
    return maks

**Menyatukan data menjadi path atau satuan dan memberikan label**

In [ ]:
def make_train_data(maks,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,maks)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))

**Membuat train data path dan penamaan**

In [ ]:
make_train_data('pisang_matang', pisang_matang)
print(len(X))

make_train_data('pisang_mentah', pisang_mentah)
print(len(X))

**visualisasi sampel data gambar beserta label**

In [ ]:
fig,ax=plt.subplots(12,2)
fig.set_size_inches(20,20)
for i in range(12):
    for j in range (2):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Dataset: '+Z[l])
        
plt.tight_layout()

**Membuat pangilan singkat untuk variable**

In [ ]:
le = LabelEncoder()
Y = le.fit_transform(Z)
Y = to_categorical(Y,20)
X = np.array(X)
X = X/255

**Membagi dataset kedalam data train dan data test split data**

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

**Mengambil data secara acak yang digunakan pada train dan test data**

In [ ]:
np.random.seed(20)
rn.seed(20)
tf.random.set_seed(20)

**Membuat model data CNN untuk sequential tabel atau summary data**

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (17,17),padding = 'Same',activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 
model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(20, activation = "softmax"))

**Membuat val akurasi pada saat epoch data**

In [ ]:
from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

**Membuat Data Generator untuk Augmentasi Data**

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # secara acak merotasi gambar 10 derajat(degrees, 0 to 180)
        zoom_range = 0.1, # secara acak  zoom atau memperbesar gambar 10 persen 
        width_shift_range=0.2,  # secara acak menggeser gambar horizontal 20 persen
        height_shift_range=0.2,  # secara acak menggeser gambar vertikal 20 persen
        horizontal_flip=True,  # secara acak membalik gambar
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

**Memberikan keterangan loss akurasi dan metric akurasi**

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

**Proses Train Data**

In [ ]:
batch = 20
epoch = 50
step = np.ceil(x_train.shape[0] / batch)
History = model.fit_generator(
    datagen.flow(x_train,y_train, batch_size = batch),
    epochs = epoch,
    validation_data = (x_test,y_test),
    verbose = 1,
    steps_per_epoch = step
)

**Menyimpan hasil model ke format .h5**

In [ ]:
model.save('/kaggle/working/modeldeteksikematanganpisang.h5')
%cd /kaggle/working/

**Konversi model h5 ke TFlite**

In [ ]:
saved_model_dir = '/kaggle/working/'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

target_names = ['pisang_matang','pisang_mentah']
labels = '\n'.join(target_names)

with open('labels.txt', 'w') as f:
  f.write(labels)

**Menampilkan squential tabel**

In [ ]:
model.summary()

**Menampilkan *Confusion Matrix***

In [ ]:
Y_pred = model.predict_generator(x_test)
y_pred = np.argmax(Y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)

**Menampilkan *Classification Report***

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

**Menampilkan *Normalized Confusion matrix* dalam bentuk tabel**

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):

    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    plt.figure(figsize=(15,15))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm, target_names, title='Confusion Matrix')

**Memanggil model .h5 yang telah ditraining sebelumnya**

In [ ]:
new_model = tf.keras.models.load_model('/kaggle/working/modeldeteksikematanganpisang.h5')

**Menampilkan sequential tabel**

In [ ]:
new_model.summary()

**Menampilkan grafik Loss**

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

**Menampilkan grafik Accuracy**

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

**Membuat variabel untuk memulai prediksi**

In [ ]:
pred=new_model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

**Menyimpan beberapa indeks yang salah diklasifikasikan**

In [ ]:
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

**Memanggil dan menampilkan gambar yang sudah di prediksi**

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(20,20)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
      
        count+=1

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(20,20)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[mis_class[count]])
        plt.tight_layout()
        count+=1

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(20,20)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Detection : "+str(le.inverse_transform([pred_digits[prop_class[count]]])))
        plt.tight_layout()
        count+=1

In [ ]:
#new_model.fit(x_train,y_train,epochs=epoch,batch_size=batch,validation_data = (x_test,y_test))